In [ ]:
from autumn.core.project import get_project
project = get_project("sm_covid2", "france")

In [ ]:
import summer2
# import numpy as np
# import pandas as pd
from autumn.core.project import get_project, load_timeseries

In [ ]:
project = get_project("sm_covid2", "france")
default_params = project.param_set.baseline
m = project.build_model(default_params.to_dict()) 

In [ ]:
# Targets represent data we are trying to fit to
from estival import targets as est

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

In [ ]:
# Import nevergrad
import nevergrad as ng

# Import our convenience wrapper
from estival.optimization.nevergrad import optimize_model

In [ ]:
death_target = project.calibration.targets[0]
sero_target = project.calibration.targets[1]

In [ ]:
targets = [
    est.NormalTarget(name="infection_deaths", data=death_target.data, stdev=1000)
]

In [ ]:
def get_estival_uniform_priors(autumn_priors):
    estival_priors = []
    for prior_dict in autumn_priors:
        assert prior_dict["distribution"] == "uniform", "Only uniform priors are currently supported"
        estival_priors.append(
            esp.UniformPrior(prior_dict["param_name"], prior_dict["distri_params"]),
        ) 
    return estival_priors
    

In [ ]:
priors = get_estival_uniform_priors(project.calibration.all_priors)

In [ ]:
bcm = BayesianCompartmentalModel(m, default_params.to_dict(), priors, targets)

In [ ]:
# TwoPointsDE is a good suggested default for some of our typical use cases
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class)

In [ ]:
# You can also suggest starting points for the optimization (as well as specify an init method for unsuggested points)
# This is the "midpoint" method by default (ie the 0.5 ppf of the prior distribution)
orunner = optimize_model(bcm, opt_class=opt_class, suggested=default_params.to_dict(), init_method="midpoint")

In [ ]:
for i in range(8):
    # Run the minimizer for a specified 'budget' (ie number of evaluations)
    rec = orunner.minimize(100)
    # Print the loss (objective function value) of the current recommended parameters
    print(rec.loss)

In [ ]:
mle_params = rec.value[1]

In [ ]:
res = bcm.run(mle_params)

In [ ]:
target = "infection_deaths"

# You can access the targets from the BCM
bcm.targets[target].data.plot(style='.')
res.derived_outputs[target].plot()